In [1]:
import bs4 as bs
import pickle
import urllib
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
import traceback

def get_soup(source_address, timeout=10):
    hdr = {'User-Agent': 'Mozilla/5.0'}

    try:
        req = urllib.request.Request(source_address, headers=hdr)
        source = urllib.request.urlopen(req, timeout=timeout).read()
        soup = bs.BeautifulSoup(source, 'lxml')
        return soup
    except urllib.error.URLError as e:
        print('Could not complete urllib request.')
        print(f"Error: {e.reason}")
        traceback.print_exc()
    except Exception as e:
        print('Could not create soup.')
        print(f"Error: {str(e)}")
        traceback.print_exc()
    
    return None

In [3]:
def read_pickle(file):
    with open(file, 'rb') as handle:
         load = pickle.load(handle)
    return load

def to_pickle(var, dir):
    directory = dir + '.pickle'
    with open(directory, 'wb') as handle:
        pickle.dump(var, handle)
    return

In [4]:
def get_links(query):
    query = query.replace(' ', '%20')
    url = 'https://medium.com/search?q=' + query

    path = "/Users/leeeunji/workspace_selenium/chromedriver"
    driver = webdriver.Chrome(path)
    driver.get(url)
    time.sleep(1)

    for i in range(0,60):
        driver.execute_script('window.scrollBy(0, 1600)')
        time.sleep(1)
    
    time.sleep(2)

    source = driver.page_source
    soup = bs.BeautifulSoup(source, 'lxml')

    a_tags = soup.find_all('a', attrs={'aria-label': 'Post Preview Title'})
    links = []
    for a in a_tags:
        try:
            href = urllib.parse.quote(a['href'], safe=':/?&=')  # URL 인코딩
            full_url = "https://medium.com" + href
            print(full_url)
            links.append(full_url)
        except Exception as e:
            print(f"Error processing URL: {a['href']}")
            print(e)
        
    driver.close()
    return links

In [5]:
def get_stories(links):
    stories = []
    for index, link in enumerate(links):
        try:
            print(f'link: {link}')
            soup = get_soup(link)
            p_tags = soup.find_all('p', class_=lambda value: value and 'pw-post-body-paragraph' in value.split())
            print(p_tags)
            story = ' '.join([p.get_text(strip=True) for p in p_tags])
            print('Story:', story)
            stories.append(story)
            print(index, 'of', len(links), 'stories')
        except:
            pass
    return stories

In [6]:
queries = ['개념']

story_dic = dict.fromkeys(queries)

In [7]:
for query in queries:
    print(' ::', query, ':: ')
    links = get_links(query)
    stories = get_stories(links)
    to_pickle(stories, 'data/medium_stories/' + query.replace(' ', '_'))
    story_dic[query] = stories

df = pd.DataFrame({col: pd.Series(stories) for col, stories in story_dic.items()})
df.columns = [col.replace(' ', '_') for col in df.columns]
df.to_csv('huge_medium.csv')

 :: 개념 :: 
https://medium.com/harrythegreat/aws-%EA%B0%80%EC%9E%A5%EC%89%BD%EA%B2%8C-vpc-%EA%B0%9C%EB%85%90%EC%9E%A1%EA%B8%B0-71eef95a7098?source=search_post---------0----------------------------
https://medium.com/%40mjspring/%EC%9D%B4%EB%B2%A4%ED%8A%B8-%EC%86%8C%EC%8B%B1-event-sourcing-%EA%B0%9C%EB%85%90-50029f50f78c?source=search_post---------1----------------------------
https://medium.com/harrythegreat/android-navigation-component-%EA%B0%9C%EB%85%90%EA%B3%BC-%ED%8A%9C%ED%86%A0%EB%A6%AC%EC%96%BC-1-5ac6ac081643?source=search_post---------2----------------------------
https://medium.com/%40youngji/%EC%A0%84%EC%9E%90%EA%B8%B0-electromagnetic-%EC%9D%98-%EA%B8%B0%EC%B4%88%EA%B0%9C%EB%85%90-f40c0e83eb4f?source=search_post---------3----------------------------
https://medium.com/one-asia/neirongdianshang-f31aae24b04d?source=search_post---------4----------------------------
https://medium.com/%40youngji/%EB%B6%80%EB%8F%99%EC%82%B0%EC%9D%98-%EA%B0%9C%EB%85%90-%EB%B0%8F-%EC%A2%85%EB%A5%98-15

In [8]:
lst = read_pickle('data/medium_stories/개념.pickle')

In [9]:
lst[0]

'주요 VPC개념을 중점으로 개발하며 머리끝까지 화가난건 손에 꼽는것같습니다. 수십번도 더 설정한 VPC인데… 네트워크 구조를 설명못한걸로 보안그룹은아냐 서브넷은아냐 소리를 들으며 순간 살의를 느껴봤습니다. 최소한 다른사람들은 저처럼 기분나쁜일은 없으면해서 최대한 쉽게 VPC를 설명해보고싶습니다. VPN은 한국어로 “가상사설망"이라합니다. 앞에 “가상"이라는 단어에서 알 수 있듯 실제 사설망이 아닌가상의 사설망입니다. 만약위 그림과같이 회사의 네트워크가 구성되어있고 보안상의 이유로 직원간 네트워크를 분리하고싶다면 기존 인터넷선 선공사도 다시해야하고 건물의 내부선을 다 뜯어고쳐야하며 다시 전용선을 깔아주어야합니다. 이를위해 가상의 망 VPN을 사용하게됩니다. VPN은 네트워크A와 네트워크B가 실제로 같은 네트워크상에 있지만 논리적으로 다른네트워크인것처럼 동작합니다. 이를 우리는 ‘가상사설망\'이라고합니다. VPC가 없다면 EC2 인스턴스들이 서로 거미줄처럼 연결되고 인터넷과 연결됩니다. 이런 구조는 시스템의 복잡도를 엄청나게 끌어올릴뿐만 아니라 하나의 인스턴스만 추가되도 모든 인스턴스를 수정해야하는 불편함이 생깁니다. 마치 인터넷 전용선을 다시까는것과 같습니다. VPC를 적용하면 위 그림과같이 VPC별로 네트워크를 구성할 수 있고 각각의 VPC에따라 다르게 네트워크 설정을 줄 수 있습니다. 또한 각각의 VPC는 완전히 독립된 네트워크처럼 작동하게됩니다. VPC를 구축하기위해서는 VPC의 아이피범위를 RFC1918이라는 사설아이피대역에 맞추어 구축해야합니다. 사설아이피란 무엇일까요? 인터넷을 위해 사용하는것이 아닌 우리끼리 사용하는 아이피주소 대역입니다. 예를들어보겠습니다. 누군가 “안방에서 리모컨좀 가져다달라”하면 저는 옆집을 가는게 아닌 우리집에 있는 “안방”으로 찾아갑니다. 안방이 프라이빗아이피(사설아이피) 우리집 주소가 퍼블릭아이피입니다. 옆집도 안방이 있고 우리집도 안방이 있지만 서로 안방을 들었을때 헷갈리지 않습니다. “안방”, “작은방”, “큰방”처럼